In [ ]:
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict
import networkx as nx
from networkx.algorithms import bipartite

**PUNKT 1**
```
Ranking    Wyraz(str).     F[procent] - czestotliwosc.      r x f(słupkami poziomymi)
```
Manuskrypt wojnicza oraz dla dowolnego języka naturalnego



Wybranie odpowiedniego tekstu do przetwarzania. str: [1- 118]


In [ ]:
with open("FSG.txt", "r") as file, open('wynik.txt', 'w') as destination:
  for idx, line in enumerate(file):
      if "page" in line or ("#" not in line and "$" not in line):
        destination.write(line)
      if idx == 2679: break

Stworzenie listy zawierającej same słowa
i wyświetlenie ilości słów

In [ ]:
raw_text = []
with open("wynik.txt", "r") as wynik:
  for line in filter(lambda l: "#" not in l, wynik):
    words: List[str] = line.strip().split(',')
    if '-' in words[-1]  or '=' in words[-1]:
      words[-1] = words[-1].replace("=",'').replace("-",'')
    raw_text.extend(word for word in words if word)



Policzenie ilości danego słowa w tekscie

In [ ]:
counted: List[Tuple[str, int]] = [(unique_word, raw_text.count(unique_word)) for unique_word in filter(lambda word: word, set(raw_text))]

In [ ]:
counted.sort(key=lambda x: -x[1])

In [ ]:
print("rank|word|ratio|chart")
ytics: List[str] = []
ratios: List[float] = []
for idx, (word, occurences) in enumerate(counted, 1):
  ratio: float = occurences/len(raw_text)*100
  ytics.append(f'{idx} | {word} | {ratio} | ')
  ratios.append(ratio*idx)

plt.figure(figsize=(30, 650))
plt.barh(ytics, ratios)
# plt.xlim([0,5])
plt.grid()
plt.xlabel("r x f")
plt.show()


**PUNKT 2**

Sąsiedztwo między wyrazami
Sąsiedztwo to dwa wyrazy do tylu i dwa wyrazy do przodu.
```
W porządku alfabetycznym
Wyr1.           Wyr1
Wyr2.           Wyr2
Wyr3.           Wyr3
Wyr4.           Wyr4
Wyr5.           Wyr5
```
I robimy strzaleczkami połączenia między wyrazami sąsiadującymi.

Powinien się znaleźć wyraz który ma znacznie dużo sąsiadów - to jest rdzeń języka.



In [ ]:
unique_words: List[str] = [word for word, _ in counted]


# dla kazdego wyrazu z listy unique_words musimy znalezc ten wyraz w licie raw_text i do niego dopisac sasiadow
nested_structure: Dict[str, Dict[str, int]] = {}
plain_structure: Dict[str, int] = {unique_word: 0 for unique_word in unique_words}
for idx, word in enumerate(raw_text[:-2]):
    for neigh_idx, neigh in enumerate(raw_text[idx:idx+3]):
        a, b = (word, neigh) if word<neigh else (neigh, word)
        if not a in nested_structure:
            nested_structure[a] = {}
        if not b in nested_structure[a]:
            nested_structure[a][b] = 0
        nested_structure[a][b] += 1
        plain_structure[a]+=1
        plain_structure[b]+=1
        

In [ ]:
N: int = 20
links_threshold: int = 10

G = nx.Graph()
G.add_nodes_from([node for node, links_count in plain_structure.items() if links_count > links_threshold], bipartite=0)
G.add_nodes_from([node+' ' for node, links_count in plain_structure.items() if links_count > links_threshold], bipartite=1)
G.add_edges_from((unique_word, neighbour+" ") for unique_word, neighbours in nested_structure.items() for neighbour, count in neighbours.items() if count > N)


is_bipartite: bool = bipartite.is_bipartite(G)

In [ ]:
plt.hist([count for neighbours in nested_structure.values() for count in neighbours.values()], bins=500)
plt.xlim([0, 20])

In [ ]:
print(max(plain_structure, key = lambda key: plain_structure[key]))

In [ ]:
plt.figure(figsize=(200,200)) #???
nx.draw_networkx(G, pos = nx.drawing.layout.bipartite_layout(G, [word for word in unique_words]), width = 2, font_size=30)

**Punkt 3** : Analiza bigramów

Badać bigramy w ramach akapitów.
```
Wyr1 wyr2 wyr3 wyr4 wyr5 wyr6 ….

Pary sąsiadujących ze sobą wyrazów -2 sąsiednie.

Wyr1   wyr2      [ilosc wystąpień]
Wyr2   wyr3      [ilosc wystąpień]
Wyr3   wyr4      [ilosc wystąpień]
Wyr4   wyr5      [ilosc wystąpień]
```
W manuskrypie większość bigramów będą pojawiające się 1 raz.




In [ ]:
zle_akapity = 0
akapit_poprawny = 0
structure1 = [[]]

with open("wynik.txt", "r") as wynik:
  for line in wynik:
    words = line.split(',')
    if len(words) == 1  and ('=' in words[0] or '#'in words[0]): zle_akapity+=1
    else: 
      if('=' in words[-1]):
        words[-1] = words[-1].replace('=','')
        structure1[akapit_poprawny] += words
        akapit_poprawny+=1
        structure1.append([])
      else: structure1[akapit_poprawny] += words

# wszystkie pary słow wg jednego akapitu  -  Nie ma par typu [ostatnie słowo z akapitu, pierwsze słowo z akapitu nastepnego]
pary_slow = []

for i in structure1 :
  for j in range(len(i)-1) :
    if( '-'  in i[j] ): i[j]= i[j].replace('-','')
    if( '-'  in i[j+1] ): i[j+1] = i[j+1].replace('-','')
    if( '\n'  in i[j] ): i[j] = i[j].replace('\n','')
    if( '\n'  in i[j+1] ): i[j+1] = i[j+1].replace('\n','')
    pary_slow.append([i[j], i[j+1]])

pary_slow
print("Ilość par słow wg jednego akapitu: ", len(pary_slow))

# unikalna lista par
unique_pary_slow = []
for i in pary_slow:
    if i not in unique_pary_slow:
        unique_pary_slow.append(i)
print("Ilość unikalnych par słow: " , len(unique_pary_slow))


counter = 0
for i in unique_pary_slow:
  for j in pary_slow:
    if(i == j): counter += 1
  i.append(counter)
  counter = 0

# posortowanie po ilosci wystapien
unique_pary_slow.sort(key = lambda i: -i[2])

# wyswietlenie wyniku
for i in unique_pary_slow:
  print(i[0],"   ", i[1], "  ilośc wystapień: ", i[2])
